In [28]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import seaborn as sns
import numpy as np
import tensorflow as tf
import pandas as pd
import os

In [31]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")

def int_to_dummies(i):
    # Create a label for the number given
    #Input: int The value of the label
    result = [0]*30
    result[int(i)] = 1
    return np.array(result)

def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model


def prediction_logits (y,batch_size,nb_label):
    # Normalize the batch of label to obtain a percentage
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    #Out : Numpy array [batch_size][10] the input normalized
    
    y_sum = np.sum(y, axis=1)
    for i in range (0,batch_size):
        for j in range (0,nb_label):
            if (y_sum[i] != 0):
                y[i][j] = y[i][j]/y_sum[i]
    
    return y


In [32]:
df = pd.read_csv("parsed_data.csv")
df = df[df.ID_FTR != -1]
target = df["ID_FTR"]
features = df[df.columns[7:]]

In [33]:
features

,AAG,AAS,AASO,AAF,AAC,AAY,AAR,HAG,HAS,HASO,HAF,HAC,HAY,HAR
0,0.235243,0.266572,-0.346070,-0.780578,2.451612,-0.994774,-0.580106,-0.962820,0.382529,0.074035,0.052537,-0.837804,-1.454220,-0.576492
1,-0.564010,-0.170714,-0.995201,0.056229,-0.109168,-0.994774,-0.580106,-0.165427,0.049468,0.074035,0.052537,0.978304,-1.454220,-0.576492
2,1.034495,2.015716,1.384945,1.032504,0.256657,-0.994774,-0.580106,-0.165427,-1.393795,-0.794851,-0.798350,-1.564248,-1.936414,-0.576492
3,-0.564010,0.922501,0.086683,0.893036,-0.109168,-1.478684,-0.580106,0.233269,-1.060734,-0.577630,1.612498,-1.564248,0.956749,-0.576492
4,-0.164384,0.047929,0.086683,0.614100,0.622483,1.424779,-0.580106,-1.361516,-1.171754,-1.880959,0.619796,-0.111361,-0.007638,-0.576492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,-0.963637,-0.061392,0.086683,-0.362175,-0.474994,-0.510863,-0.580106,1.429358,1.159671,1.811808,0.903425,0.615083,-0.007638,-0.576492
1953,-0.564010,-0.389356,-0.562447,0.753568,-0.109168,0.456958,1.142606,0.631965,1.048651,0.942921,1.187054,-0.474582,0.474556,-0.576492
1954,0.235243,-0.389356,0.086683,1.032504,-1.206645,0.456958,-0.580106,-1.361516,-0.172572,-0.577630,0.619796,-0.474582,0.474556,1.180820
1955,0.634869,0.485215,0.086683,0.893036,-0.840820,0.940868,-0.580106,-0.165427,0.604570,0.942921,-1.223794,-0.111361,-0.489832,-0.576492


In [34]:
x_test = features.iloc[1850:].values
y_test = target.iloc[1850:].values

x_train = features.iloc[:1850].values
y_train = target.iloc[:1850].values

In [35]:
x_train = x_train.astype(float)

In [37]:
model = create_model()

checkpoint_path = "models/model1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.fit(x_train,y_train, epochs=10,validation_data=(x_test,y_test),
          callbacks=[cp_callback])

Train on 1850 samples, validate on 107 samples
Epoch 1/10
1536/1850 [=======================>......] - ETA: 0s - loss: 1.1504 - accuracy: 0.3639
Epoch 00001: saving model to models/model1.ckpt
1850/1850 [==============================] - 2s 944us/sample - loss: 1.1436 - accuracy: 0.3692 - val_loss: 1.0768 - val_accuracy: 0.4766
Epoch 2/10
1248/1850 [===================>..........] - ETA: 0s - loss: 1.0824 - accuracy: 0.4335
Epoch 00002: saving model to models/model1.ckpt
1850/1850 [==============================] - 0s 100us/sample - loss: 1.0849 - accuracy: 0.4281 - val_loss: 1.0666 - val_accuracy: 0.4766
Epoch 3/10
1632/1850 [=========================>....] - ETA: 0s - loss: 1.0691 - accuracy: 0.4375
Epoch 00003: saving model to models/model1.ckpt
1850/1850 [==============================] - 0s 110us/sample - loss: 1.0698 - accuracy: 0.4335 - val_loss: 1.0525 - val_accuracy: 0.4953
Epoch 4/10
1440/1850 [======================>.......] - ETA: 0s - loss: 1.0631 - accuracy: 0.4465
Epoch 

In [38]:
index = np.random.randint(0,107)
print(y_test[index])
print_logits(model.predict(x = x_test[index].reshape(1,14), batch_size = 1),1,3)

0
------------ 0 ------------
prédiction :  0 pourcentage de certitude : 22.5  %
prédiction :  1 pourcentage de certitude : 22.3  %
prédiction :  2 pourcentage de certitude : 55.2  %


In [24]:
y_pred = model.predict(x = x_train)
y_pred = np.argmax(y_pred, axis=1)
conf_mat = tf.math.confusion_matrix(y_train, y_pred)

(1, 14)